# Fine-tune Llama2 using QLoRA and Deploy Model with Multiple Adapters

In this tutorial, we will fine-tune a llama2 model using QLoRA, optimize it using ONNX Runtime tools, and extract the fine-tuned adapters from the model. 
The resulting model can be deployed with multiple adapters for different tasks.

## Prerequisites

Before running this tutorial, please ensure you already installed olive-ai. Please refer to the [installation guide](https://github.com/microsoft/Olive?tab=readme-ov-file#installation) for more information.

### Install Dependencies
We will optimize for `CUDAExecutionProvider` so the corresponding `onnxruntime` should also be installed allong with the other dependencies:

In [ ]:
!pip install -r requirements-qlora.txt
!pip install ipywidgets tabulate

### Get access to model and fine-tuning dataset

Get access to the following resources on Hugging Face Hub:
- [meta-llama/Llama-2-7b-hf](https://huggingface.co/meta-llama/Llama-2-7b-hf)
- [nampdn-ai/tiny-codes](https://huggingface.co/nampdn-ai/tiny-codes)

Login to your Hugging Face account:

In [ ]:
from huggingface_hub import login

login()

## Workflow

The olive workflow is defined in the [llama2_qlora.json](../../llama2_qlora.json) file. 

It fine-tunes [meta-llama/Llama-2-7b-hf](https://huggingface.co/meta-llama/Llama-2-7b-hf) model using [QLoRA](https://arxiv.org/abs/2305.14314) on a subsection of [nampdn-ai/tiny-codes](https://huggingface.co/nampdn-ai/tiny-codes) to generate python code given a prompt. The fine-tuned model is then optimized using ONNX Runtime Tools.

Performs optimization pipeline:
- GPU, FP16: *Pytorch Model -> Fine-tuned Pytorch Model -> Onnx Model -> Transformers Optimized Onnx Model fp16 -> Extract Adapters*

**Note:**
- The code language is set to `Python` but can be changed to other languages by changing the `language` field in the config file.
Supported languages are Python, TypeScript, JavaScript, Ruby, Julia, Rust, C++, Bash, Java, C#, and Go. Refer to the [dataset card](https://huggingface.co/datasets/nampdn-ai/tiny-codes) for more details on the dataset.
- The `ExtractAdapters` pass in this workflow extracts the lora adapters from the model and converts them into model inputs. An alternate option is to extract them as external intializers. This can be done by setting `"make_inputs": false` in the `ExtractAdapters` pass configuration.

Run the worklow:

In [ ]:
!olive run --config llama2_qlora.json --setup

[2024-04-25 12:53:57,682] [INFO] [run.py:279:run] test/lib/python3.8/site-packages/olive/olive_config.json
[2024-04-25 12:53:57,683] [INFO] [run.py:285:run] Loading run configuration from: llama2_qlora.json
[2024-04-25 12:53:58,125] [INFO] [run.py:91:dependency_setup] The following packages are required in the local environment: ['onnxruntime-gpu']
[2024-04-25 12:53:58,164] [INFO] [run.py:321:check_local_ort_installation] onnxruntime-gpu is already installed.


In [ ]:
!CUDA_VISIBLE_DEVICES=0 olive run --config llama2_qlora.json

[2024-04-25 12:59:55,351] [INFO] [run.py:279:run] Loading Olive module configuration from: test/lib/python3.8/site-packages/olive/olive_config.json
[2024-04-25 12:59:55,351] [INFO] [run.py:285:run] Loading run configuration from: llama2_qlora.json
[2024-04-25 12:59:56,812] [INFO] [accelerator_creator.py:224:create_accelerators] Running workflow on accelerator specs: gpu-cuda
[2024-04-25 12:59:56,812] [INFO] [engine.py:107:initialize] Using cache directory: cache
[2024-04-25 12:59:56,813] [INFO] [engine.py:263:run] Running Olive on accelerator: gpu-cuda
[2024-04-25 12:59:56,813] [INFO] [engine.py:1075:_create_system] Creating target system ...
[2024-04-25 12:59:56,813] [INFO] [engine.py:1078:_create_system] Target system created in 0.000093 seconds
[2024-04-25 12:59:56,813] [INFO] [engine.py:1087:_create_system] Creating host system ...
[2024-04-25 12:59:56,813] [INFO] [engine.py:1090:_create_system] Host system created in 0.000076 seconds
[2024-04-25 12:59:56,862] [INFO] [engine.py:325

The output model files are can be found at:
- Model: `models/tiny-codes-qlora/qlora-conversion-transformers_optimization-extract/gpu-cuda_model/model.onnx`
- Adapter weights: `models/tiny-codes-qlora/qlora-conversion-transformers_optimization-extract/gpu-cuda_model/adapter_weights.npz`

## Export Pre-existing Adapters

Olive provides a standalone script to export the fine-tuned adapters from a pre-existing repository on huggingface hub or your local directory. The adapters must be fine-tuned on the same base model with the same configuration as the model obtained from the previous step. 

Lets export the adapters from [Mikael110/llama-2-7b-guanaco-qlora](https://huggingface.co/Mikael110/llama-2-7b-guanaco-qlora):

In [ ]:
# run this cell to see the available options to export-adapters command
!olive export-adapters --help

In [ ]:
!olive export-adapters --adapter_path Mikael110/llama-2-7b-guanaco-qlora --output_path models/exported/guanaco_qlora.npz --pack_weights --dtype float16

## Deploy Model with Multiple Adapters

We can now deploy the same base model with multiple adapters for different tasks by loading the adapter weights independently of the model and providing the relevant weights as input at inference time.

We implemented an example class `ORTGenerator` in [generator.py](../utils/generator.py) that loads the model and adapters, and generates code given a prompt. If your execution provider supports IO Binding, it is recommended to use it for better performance since the adapter weights will be preallocated in the device memory.

In [ ]:
import sys
from pathlib import Path

# add the utils directory to the path
sys.path.append(str(Path().resolve().parent / "utils"))

from generator import ORTGenerator
from transformers import AutoTokenizer

base_model_name = "meta-llama/llama-2-7b-hf"
model_path = "models/tiny-codes-qlora/qlora-conversion-transformers_optimization-extract/gpu-cuda_model/model.onnx"
adapters = {
    "guanaco": {
        "weights": "models/exported/guanaco_qlora.npz",
        "template": "### Human: {prompt} ### Assistant:"
    },
    "tiny-codes": {
        "weights": "models/tiny-codes-qlora/qlora-conversion-transformers_optimization-extract/gpu-cuda_model/adapter_weights.npz",
        "template": "### Question: {prompt} \n### Answer:"
    }
}

# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# load the generator
generator = ORTGenerator(model_path, tokenizer, execution_provider="CUDAExecutionProvider", device_id=0, adapters=adapters, adapter_mode="inputs")

### Generate using Guanaco Adapters

In [ ]:
prompt = "What time is it?"

response = generator.generate(prompt, adapter="guanaco", max_gen_len=100, use_io_binding=True)

print(response)

### Human: What time is it? ### Assistant: I'm sorry, but as an AI language model, I do not have access to real-time information.

However, I can try to estimate the current time based on the context of your question and my knowledge of the current time zone.

In general, the current time can vary depending on your location and the time zone you are in.

If you would like to know the current time for a specific location, you can try searching for the time zone for that


### Generate with Tiny Codes Adapters

In [ ]:
prompt = "Calculate the sum of a list of integers."

response = generator.generate(prompt, adapter="tiny-codes", max_gen_len=100, use_io_binding=True)

print(response)

### Question: Calculate the sum of a list of integers. 
### Answer: Here is some python code that implements this functionality: 

def sum_list(lst):
    """
    Calculates the sum of a list of integers.
    
    Args:
        lst (list): List of integers to sum.
    
    Returns:
        int: Sum of the list elements.
    """  
    
    return sum(lst)
    
    
if __name__ == "__main__":
   
